# Mayall Primary Mirror Wash Data

## Setup

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import gspread as gs
import matplotlib.cm as cm
from matplotlib.gridspec import GridSpec
import ipywidgets as widgets
from IPython.display import display, clear_output
from sklearn.linear_model import LinearRegression

In [2]:
import matplotlib as mpl

mpl.rcParams['font.family'] = 'sans-serif'
colors = mpl.colormaps['tab20']

/var/folders/my/l9_wdk6j0g186w61vqtc_cw40000gp/T/ipykernel_94596/3991778654.py:4: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  colors = mpl.colormaps['tab20']


In [3]:
mayall_data = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vSWF3pE1qBNcatq1Cm-H2z6mAGBPA-EbyUhujoOHXUU9-BfGYkmzP0YaFslu0bm2efn6AB9fNwUEUHj/pub?gid=0&single=true&output=csv')
wavelengths = mayall_data.columns[9:]
mayall_data.index = pd.to_datetime(mayall_data['Date'])
mayall_data.drop('Date',axis=1, inplace=True)

In [4]:
HCL_STAND_SCI=np.array([ 86.79,  86.88,  86.94,  86.97,  87.  ,  86.99,  86.97,  86.94,
                     86.89,  86.85,  86.8 ,  86.73,  86.65,  86.57,  86.47,  86.36,
                     86.24,  86.12,  85.98,  85.83,  85.68,  85.51,  85.33,  85.14,
                     84.95,  84.75,  84.54,  84.33,  84.1 ,  83.86,  83.61])

standard_sci = pd.DataFrame(HCL_STAND_SCI).T
standard_sci.columns = wavelengths

In [5]:
wash_types = list(np.unique(mayall_data['Wash Type']))
wash_types.append('All')
coatings = list(np.unique(mayall_data['Coating Date']))
coatings.append('All')

In [6]:
def get_dates(wash_type):
    if wash_type == 'All':
        dates = np.unique(mayall_data.index.strftime('%Y-%m-%d'))
    else:
        mask = mayall_data['Wash Type'] == wash_type
        dates = np.unique(mayall_data[mask].index.strftime('%Y-%m-%d'))
    return dates[::-1]
def get_dates2(coating):
    if coating == 'All':
        dates = np.unique(mayall_data.index.strftime('%Y-%m-%d'))
    else:
        mask = mayall_data['Coating Date'] == coating
        dates = np.unique(mayall_data[mask].index.strftime('%Y-%m-%d'))
    return dates[::-1]

In [7]:
# Daily Plot Widgets
wash_button = widgets.RadioButtons(
    options=wash_types,
    value='All',
    description='Wash Type:',
    disabled=False
)

date_selector = widgets.Dropdown(
    options=get_dates('All'),
    description='Date:',
    disabled=False,
)

daily_plot_button = widgets.Button(
    description='Daily Plot',
    disabled=False,
    button_style='', 
    tooltip='Generate daily plot',
    icon='check'
)

output = widgets.Output()

# Callback for button click
def on_daily_plot_click(b):
    with output:
        clear_output(wait=True)
        daily_plot(date_selector.value)
        plt.show()

daily_plot_button.on_click(on_daily_plot_click)

# Callback for wash type change
def on_wash_type_change(change):
    if change['name'] == 'value':
        # Update the dates dropdown options based on selected wash type
        date_selector.options = get_dates(change['new'])

wash_button.observe(on_wash_type_change, names='value')

In [22]:
# Widgets for Change Plot
coatings_selection = widgets.RadioButtons(
    options=coatings,
    value='All',
    description='Coatings:',
    disabled=False
)

start_date = widgets.Dropdown(
    options=get_dates('All'),
    value=get_dates('All')[-1],  # Usually latest date
    description='Start Date:',
    disabled=False,
)

end_date = widgets.Dropdown(
    options=get_dates('All'),
    value=get_dates('All')[0],  # Usually earliest date
    description='End Date:',
    disabled=False,
)

change_plot_button = widgets.Button(
    description='Change Plot',
    disabled=False,
    button_style='',  # e.g. 'success', 'info', 'warning', 'danger'
    tooltip='Generate plot with selected dates',
    icon='check'
)

linreg_selection = widgets.Checkbox(
    value=False,
    description='Lin. Regression',
    disabled=False,
    indent=False
)

outliers_list = widgets.Text(
    value='2024-10-21',
    placeholder='YYYY-MM-DD',
    description='Outliers:',
    disabled=False
)

output2 = widgets.Output()

# Button click handler
def on_change_plot_click(b):
    with output2:
        clear_output(wait=True)
        change_plot(start_date.value, end_date.value)
        plt.show()

change_plot_button.on_click(on_change_plot_click)

def on_coatings_selection_change(change):
    if change['name'] == 'value':  # Only respond to value changes
        dates = get_dates2(change['new'])
        print(dates)
        start_date.options = dates
        start_date.value = dates[-1]
        end_date.options = dates
        end_date.value=dates[0]

coatings_selection.observe(on_coatings_selection_change, names='value')


## Code

In [9]:
def reduce_data(df_, sc='SCI', wash='AW'):
    """
    Reduce spectral data by normalizing measurement data with calibration standards.

    Parameters:
    - df_: pandas DataFrame containing spectral data
    - sc: string, either 'SCI' or 'SCE' specifying the measurement type
    - wash: string, specifying wash type, e.g. 'AW' or 'BW'

    Returns:
    - reduced_data: normalized measurement means (pandas Series/DataFrame)
    - measurement_sem: measurement standard error means (pandas Series/DataFrame)
    """

    # Group by relevant categories and calculate aggregations
    grouped = df_.groupby(['Type', 'BW/AW', 'Tag'])[wavelengths].agg(['mean', 'sem', 'std', 'min', 'max'])

    # Extract calibration group means for scaling
    cal_group = grouped.loc[('calibration', wash, 'SCI')].xs('mean', level=1)

    # Extract measurement group data (means and sems)
    meas_group = grouped.loc[('measurement', wash, sc)]
    meas_mean = meas_group.xs('mean', level=1)
    meas_sem = meas_group.xs('sem', level=1)

    # Calculate scale factor by normalizing calibration group against standard
    scale = cal_group / standard_sci.loc[0]

    # Normalize measurement means by scale factor
    reduced_data = meas_mean / scale

    return reduced_data, meas_sem

In [15]:
def daily_plot(date):
    """
    Plot Before and After wash data for a given date.
    Also shows the ratio between them to indicate improvement.

    Parameters:
    - date: the date to filter mayall_data on
    """

    sc_map = {'Reflectance': 'SCI', 'Scatter': 'SCE'}
    colors = {'BW': 'blue', 'AW': 'orange'}

    # Filter data for the given date
    d_ = mayall_data.loc[mayall_data.index == date]

    if d_.empty:
        print(f"No data available for date {date}")
        return

    wash_type = d_['Wash Type'].iloc[0]

    for label, sc in sc_map.items():
        fig = plt.figure(figsize=(10, 4))
        gs = GridSpec(2, 1, height_ratios=[1, 3], hspace=0.0)

        ax_ratio = fig.add_subplot(gs[0])
        ax_values = fig.add_subplot(gs[1], sharex=ax_ratio)

        # Initialize flags
        has_bw = False
        has_aw = False

        # Before Wash
        try:
            reduced_bw, bw_err = reduce_data(d_, sc, 'BW')
            reduced_bw.plot(ax=ax_values, yerr=bw_err, alpha=0.6, label='BW', color=colors['BW'])
            has_bw = True
        except Exception as e:
            print(f"No data for Before Wash ({label}): {e}")

        # After Wash
        try:
            reduced_aw, aw_err = reduce_data(d_, sc, 'AW')
            reduced_aw.plot(ax=ax_values, yerr=aw_err, alpha=0.6, label='AW', color=colors['AW'])
            has_aw = True
        except Exception as e:
            print(f"No data for After Wash ({label}): {e}")

        # Plot ratio if both are available
        if has_bw and has_aw:
            ratio = reduced_aw / reduced_bw
            ratio.plot(ax=ax_ratio, label='AW/BW', color='k', alpha=0.5)

        # Customize ratio plot
        ax_ratio.set_title(f'Mean {label} ({sc}) Curves for {date}, {wash_type} wash')
        ax_ratio.set_ylabel('Wash % Increase')
        ax_ratio.legend()
        ax_ratio.tick_params(axis='x', labelbottom=False)  # Hide x tick labels on top plot

        # Customize main plot
        ax_values.set_xlabel('Wavelength (nm)')
        ax_values.set_ylabel('Mean Value')
        ax_values.legend()

        plt.rc('font', family='Arial')
        plt.tight_layout()
        plt.show()


In [25]:
def change_plot(start_date, end_date):
    """
    Plot mean values with error bars over a date range, optionally with linear regression fits.
    Excludes outliers specified in the outliers_list widget.

    Parameters:
    - start_date: start of date range (string or datetime-like)
    - end_date: end of date range (string or datetime-like)
    """

    # Filter data within date range
    this_df = mayall_data.loc[start_date:end_date]
    dates = pd.Series(this_df.index.unique())

    # Process outliers from widget, filter them out from dates
    outliers = [o.strip() for o in outliers_list.value.split(',') if o.strip()]
    if outliers:
        mask = ~dates.astype(str).apply(lambda d: any(o in d for o in outliers))
        dates = dates[mask]

    results = []
    for date in dates:
        d_ = this_df.loc[this_df.index == date]
        try:
            reduced, err = reduce_data(d_)
        except Exception:
            reduced, err = reduce_data(d_, sc='SCI', wash='BW')
            print(f'Only data for BW on {date}')
        results.append((date, (reduced, err)))

    # Extract x (dates), y means, and y errors
    x = pd.to_datetime([r[0] for r in results])
    y_means = [r[1][0] for r in results]
    y_errs = [r[1][1] for r in results]

    # Calculate means for B, G, R segments and their errors
    def mean_segment(data_list, start, end):
        if end is None:
            return [np.mean(d.iloc[start:]) for d in data_list]
        else:
            return [np.mean(d.iloc[start:end]) for d in data_list]

    B, G, R = (mean_segment(y_means, 0, 10),
               mean_segment(y_means, 10, 20),
               mean_segment(y_means, 20, None))

    B_err, G_err, R_err = (mean_segment(y_errs, 0, 10),
                          mean_segment(y_errs, 10, 20),
                          mean_segment(y_errs, 20, None))

    # Plotting setup
    fig, ax = plt.subplots(figsize=(10, 4))
    colors = {'B': 'blue', 'G': 'green', 'R': 'red'}
    data_series = {'B': (B, B_err), 'G': (G, G_err), 'R': (R, R_err)}

    for label, (means, errs) in data_series.items():
        ax.errorbar(x, means, yerr=errs, fmt='o', mfc=colors[label], mec='black', ecolor='black')
        if linreg_selection.value:
            date_range, y_fit, r2 = lin_reg(x, (means, errs))
            ax.plot(date_range, y_fit, color=colors[label], label=f'{label} fit (R²={r2:.2f})')

    # Mark coating dates on plot
    coating_dates = pd.to_datetime(coatings[:4], format="%m/%d/%Y")
    coating_mask = (coating_dates >= pd.to_datetime(start_date)) & (coating_dates <= pd.to_datetime(end_date))
    for coating in coating_dates[coating_mask]:
        ax.axvline(coating, color='k', ls='--')

    if linreg_selection.value:
        ax.legend()

    plt.tight_layout()
    plt.show()

In [17]:
def lin_reg(x, y_with_err):
    """
    Perform weighted linear regression on dates vs values with errors.

    Parameters:
    - x: pd.Series or list of datetime-like objects (dates)
    - y_with_err: tuple (y_values, y_errors), each list-like of numeric values

    Returns:
    - date_range: pd.DatetimeIndex for plotting regression line
    - y_pred: predicted y-values from regression model
    - r2_weighted: weighted coefficient of determination (R²)
    """

    y, y_err = y_with_err
    y = np.array(y, dtype=float)
    y_err = np.array(y_err, dtype=float)

    # Compute weights as inverse variance
    weights = 1 / (y_err ** 2)

    # Prepare x for regression as ordinal numbers
    xx = pd.Series(x).map(pd.Timestamp.toordinal).values.reshape(-1, 1)

    # Fit linear regression with sample weights
    model = LinearRegression()
    model.fit(xx, y, sample_weight=weights)

    # Create date range for smooth regression line
    date_range = pd.date_range(start=min(x), end=max(x), periods=len(y))
    x_range = pd.Series(date_range).map(pd.Timestamp.toordinal).values.reshape(-1, 1)

    y_pred = model.predict(x_range)

    # Manually compute weighted R²
    ss_res = np.sum(weights * (y - model.predict(xx)) ** 2)
    ss_tot = np.sum(weights * (y - np.average(y, weights=weights)) ** 2)
    r2_weighted = 1 - ss_res / ss_tot if ss_tot != 0 else float('nan')

    return date_range, y_pred, r2_weighted


## Plots

### Daily Plot

In [18]:
display(wash_button, date_selector, daily_plot_button, output)

RadioButtons(description='Wash Type:', index=5, options=('CO2', 'No Wash', 'coating', 'contact', 'contactless'…

Dropdown(description='Date:', index=13, options=('2025-06-30', '2025-06-23', '2025-06-20', '2025-06-16', '2025…

Button(description='Daily Plot', icon='check', style=ButtonStyle(), tooltip='Generate daily plot')

Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': '<Figure size 1000x400 with 2 Axes>', '…

### Change over time

In [26]:
display(coatings_selection, start_date, end_date, linreg_selection, outliers_list, change_plot_button, output2)

RadioButtons(description='Coatings:', index=2, options=('12/12/2017', '7/15/2014', '7/20/2023', '8/1/2008', 'A…

Dropdown(description='Start Date:', index=39, options=('2025-06-30', '2025-06-23', '2025-06-20', '2025-06-16',…

Dropdown(description='End Date:', options=('2025-06-30', '2025-06-23', '2025-06-20', '2025-06-16', '2025-06-09…

Checkbox(value=True, description='Lin. Regression', indent=False)

Text(value='2024-10-21', description='Outliers:', placeholder='YYYY-MM-DD')

Button(description='Change Plot', icon='check', style=ButtonStyle(), tooltip='Generate plot with selected date…

Output(outputs=({'name': 'stdout', 'text': 'Only data for BW on 2025-03-28 00:00:00\nOnly data for BW on 2025-…

In [ ]:
# Download a CSV
# For regression plot: write out coefficients
# Determine how long it would take to get to threshold